<a href="https://colab.research.google.com/github/leniartemily/QB1/blob/main/proday_NFL_EL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,326 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,428 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRele

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UDF").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType


In [4]:
from google.colab import files

uploaded = files.upload()

Saving NFL_QB_Proday_Starters.csv to NFL_QB_Proday_Starters.csv


In [5]:
import pandas as pd
import io
  
df = pd.read_csv('NFL_QB_Proday_Starters.csv', header= 0,
                        encoding= 'unicode_escape')
df.head()

,ï»¿Year,Name,College,POS,ProDay/Combine,Height_In,Weight_In,Wonderlic,40_Yard,BenchPress,VertLeap_In,BroadJump_In,Shuttle,3Cone,Started
0,2021,Ian Book,Notre Dame,QB,ProDay,72.00,211,NaN,4.65,NaN,32.5,115,4.20,7.00,No
1,2021,Shane Buechele,Southern Methodist (TX),QB,ProDay,72.25,210,NaN,4.96,NaN,30.0,114,4.60,7.34,No
2,2021,K.J. Costello,Mississippi State,QB,ProDay,76.50,225,NaN,5.11,NaN,27.5,103,4.75,NaN,No
3,2021,Sam Ehlinger,Texas,QB,ProDay,73.13,220,NaN,4.84,NaN,36.5,114,4.44,7.15,No
4,2021,Justin Fields,Ohio State,QB,ProDay,74.75,227,NaN,4.46,NaN,NaN,119,NaN,NaN,Yes


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ï»¿Year         70 non-null     int64  
 1   Name            70 non-null     object 
 2   College         70 non-null     object 
 3   POS             70 non-null     object 
 4   ProDay/Combine  70 non-null     object 
 5   Height_In       70 non-null     float64
 6   Weight_In       70 non-null     int64  
 7   Wonderlic       0 non-null      float64
 8   40_Yard         66 non-null     float64
 9   BenchPress      6 non-null      float64
 10  VertLeap_In     65 non-null     float64
 11  BroadJump_In    70 non-null     int64  
 12  Shuttle         64 non-null     float64
 13  3Cone           63 non-null     float64
 14  Started         70 non-null     object 
dtypes: float64(7), int64(3), object(5)
memory usage: 8.3+ KB


In [7]:
df.drop(columns=["Wonderlic","BenchPress","3Cone"],axis=1, inplace=True)

In [8]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 69
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ï»¿Year         64 non-null     int64  
 1   Name            64 non-null     object 
 2   College         64 non-null     object 
 3   POS             64 non-null     object 
 4   ProDay/Combine  64 non-null     object 
 5   Height_In       64 non-null     float64
 6   Weight_In       64 non-null     int64  
 7   40_Yard         64 non-null     float64
 8   VertLeap_In     64 non-null     float64
 9   BroadJump_In    64 non-null     int64  
 10  Shuttle         64 non-null     float64
 11  Started         64 non-null     object 
dtypes: float64(4), int64(3), object(5)
memory usage: 6.5+ KB


In [9]:
df.drop(columns=["Name","ï»¿Year","POS"],axis=1, inplace=True)

In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 0 to 69
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   College         64 non-null     object 
 1   ProDay/Combine  64 non-null     object 
 2   Height_In       64 non-null     float64
 3   Weight_In       64 non-null     int64  
 4   40_Yard         64 non-null     float64
 5   VertLeap_In     64 non-null     float64
 6   BroadJump_In    64 non-null     int64  
 7   Shuttle         64 non-null     float64
 8   Started         64 non-null     object 
dtypes: float64(4), int64(2), object(3)
memory usage: 5.0+ KB


In [11]:
X = pd.get_dummies(df)
X

,Height_In,Weight_In,40_Yard,VertLeap_In,BroadJump_In,Shuttle,College_Appalachian State (NC),College_Arkansas,College_Arkansas State,College_Auburn,College_Baylor,College_Boston College,College_Brigham Young,College_Central Connecticut,College_Central Florida,College_Central Michigan,College_Cincinnati,College_Coastal Carolina (SC),College_Colorado State,College_East Carolina (NC),College_Eastern Michigan,College_Eastern Washington,College_Florida,College_Florida Atlantic,College_Florida International,College_Florida State,College_Georgetown (KY),College_Grambling State (LA),College_Hillsdale (MI),College_Illinois,College_Iowa State,College_Louisville,College_Maryland,College_Memphis,College_Miami (FL),College_Michigan,College_Mississippi State,College_Northwestern,College_Notre Dame,College_Oklahoma State,College_Pittsburgh,College_Purdue,College_Rutgers,College_Samford (AL),College_South Carolina,College_South Dakota,College_South Florida,College_Southern Methodist (TX),College_Southern Mississippi,College_Stanford,College_Syracuse,College_Temple,College_Tennessee,College_Texas,College_Tiffin (OH),College_Troy (AL),College_Tulane (LA),College_Tulsa (OK),College_Utah,College_Wake Forest,College_Washington State,College_Wisconsin,ProDay/Combine_ProDay,Started_No,Started_Yes
0,72.00,211,4.65,32.5,115,4.20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,72.25,210,4.96,30.0,114,4.60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
2,76.50,225,5.11,27.5,103,4.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
3,73.13,220,4.84,36.5,114,4.44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
5,78.63,234,4.61,32.5,117,4.22,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,75.00,225,5.13,28.5,104,4.59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
66,77.13,237,4.87,29.0,111,4.24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
67,73.88,223,4.78,29.0,105,4.40,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
68,74.38,219,4.99,29.0,105,4.46,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [12]:
X.drop(columns=["Started_No"],axis=1, inplace=True)

In [13]:
X.head()

,Height_In,Weight_In,40_Yard,VertLeap_In,BroadJump_In,Shuttle,College_Appalachian State (NC),College_Arkansas,College_Arkansas State,College_Auburn,College_Baylor,College_Boston College,College_Brigham Young,College_Central Connecticut,College_Central Florida,College_Central Michigan,College_Cincinnati,College_Coastal Carolina (SC),College_Colorado State,College_East Carolina (NC),College_Eastern Michigan,College_Eastern Washington,College_Florida,College_Florida Atlantic,College_Florida International,College_Florida State,College_Georgetown (KY),College_Grambling State (LA),College_Hillsdale (MI),College_Illinois,College_Iowa State,College_Louisville,College_Maryland,College_Memphis,College_Miami (FL),College_Michigan,College_Mississippi State,College_Northwestern,College_Notre Dame,College_Oklahoma State,College_Pittsburgh,College_Purdue,College_Rutgers,College_Samford (AL),College_South Carolina,College_South Dakota,College_South Florida,College_Southern Methodist (TX),College_Southern Mississippi,College_Stanford,College_Syracuse,College_Temple,College_Tennessee,College_Texas,College_Tiffin (OH),College_Troy (AL),College_Tulane (LA),College_Tulsa (OK),College_Utah,College_Wake Forest,College_Washington State,College_Wisconsin,ProDay/Combine_ProDay,Started_Yes
0,72.00,211,4.65,32.5,115,4.20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,72.25,210,4.96,30.0,114,4.60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,76.50,225,5.11,27.5,103,4.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,73.13,220,4.84,36.5,114,4.44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
5,78.63,234,4.61,32.5,117,4.22,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [14]:
y=X["Started_Yes"]
X.drop(columns=["Started_Yes"],axis=1, inplace=True)

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [17]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
classifier.fit(X_train, y_train)
print(classifier)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [19]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9583333333333334
Testing Data Score: 0.75


In [20]:
def getTestData(Height, Weight, FortyYard, VertLeap, BroadJump,	Shuttle, College):
  testData = [Height, Weight, FortyYard, VertLeap, BroadJump,	Shuttle, 0, 1, 0, 0]
  return testData

In [21]:
predictions = classifier.predict(new_data)
print("Players are either 0 (not drafted) or 1 (drafted)")
print(f"The player was classified as: {predictions}")

NameError: ignored